## **DATASET**
Url: https://universe.roboflow.com/socrobdatasets/merged-dataset-robocup-2024

In [1]:
!pip install torch torchvision torchaudio
!pip install requests pydantic opencv-python gdown
!pip install git+https://github.com/facebookresearch/detectron2.git

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-9zosur8l
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-9zosur8l
  Resolved https://github.com/facebookresearch/detectron2.git to commit ebe8b45437f86395352ab13402ba45b75b4d1ddb
  Preparing metadata (setup.py) ... done


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import json
import os
import subprocess

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.utils.logger import setup_logger
import torch


In [17]:
import torch  # Necesario para detectar la disponibilidad de GPU
from detectron2.data import DatasetCatalog, MetadataCatalog

class FruitSegmentation:
    """
    A class used to download, filter, modify, and train a fruit segmentation model using Detectron2.

    Attributes:
    -----------
    train_dataset_path : str
        The path to the training dataset directory.
    test_dataset_path : str
        The path to the testing dataset directory.
    output_dir : str
        The directory where the model and results will be saved.
    num_classes : int
        Number of target classes for segmentation.
    base_lr : float
        The base learning rate for the model.
    max_iter : int
        Maximum number of iterations for training.
    batch_size : int
        The batch size used for training.
    num_workers : int
        Number of worker threads used for loading data.
    device : str
        Device used for training and inference (CPU/GPU).
    classes_to_keep : list
        List of classes to keep from the dataset.
    """

    def __init__(self, train_dataset_path, test_dataset_path,
                 output_dir="segmentacion", num_classes=4, base_lr=0.0025,
                 max_iter=1000, batch_size=2, num_workers=2, device=None):
        """
        Initializes the FruitSegmentation class with dataset paths, training parameters, and filtering setup.

        Parameters:
        -----------
        train_dataset_path : str
            The path to the training dataset directory.
        test_dataset_path : str
            The path to the testing dataset directory.
        output_dir : str
            The directory where the model and results will be saved.
        num_classes : int
            Number of target classes for segmentation.
        base_lr : float
            The base learning rate for the model.
        max_iter : int
            Maximum number of iterations for training.
        batch_size : int
            The batch size used for training.
        num_workers : int
            Number of worker threads used for loading data.
        device : str, optional
            Device used for training and inference (CPU/GPU). If None, the device will be automatically chosen based on availability.
        """
        self.train_dataset_path = train_dataset_path
        self.test_dataset_path = test_dataset_path
        self.num_classes = num_classes
        self.output_dir = output_dir
        self.base_lr = base_lr
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.num_workers = num_workers

        # Inicializa la lista de clases a mantener
        self.classes_to_keep = ['apple', 'banana', 'orange', 'pear']

        # Detectar GPU si está disponible, de lo contrario usar CPU
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")

        self._download_dataset()
        self._filter_dataset_annotations(self.train_dataset_path)
        self._filter_dataset_annotations(self.test_dataset_path)
        self._clear_previous_registration()
        self._register_datasets()
        self.cfg = self._setup_cfg()

    def _clear_previous_registration(self):
        """
        Clears any previous dataset registrations to avoid metadata conflicts.
        """
        if "fruit_dataset_train" in DatasetCatalog.list():
            DatasetCatalog.remove("fruit_dataset_train")
            MetadataCatalog.remove("fruit_dataset_train")

        if "fruit_dataset_test" in DatasetCatalog.list():
            DatasetCatalog.remove("fruit_dataset_test")
            MetadataCatalog.remove("fruit_dataset_test")

    def _download_dataset(self):
        """
        Downloads the dataset from Roboflow if it's not available locally.
        """
        if not os.path.exists(self.train_dataset_path) or not os.path.exists(self.test_dataset_path):
            print("Descargando el dataset desde Roboflow...")
            subprocess.run(
                'curl -L "https://universe.roboflow.com/ds/L7xp8qMl6k?key=Npf4RqYMud" > roboflow.zip',
                shell=True)
            subprocess.run("unzip roboflow.zip", shell=True)
            os.remove("roboflow.zip")
            print("Dataset descargado, extraído y el archivo .zip eliminado.")
        else:
            print("El dataset ya está disponible.")

    def _filter_dataset_annotations(self, dataset_path):
        """
        Filters out classes and images that do not contain relevant classes.

        Parameters:
        -----------
        dataset_path : str
            Path to the dataset directory where the annotations and images are stored.
        """
        json_path = os.path.join(dataset_path, "_annotations.coco.json")

        with open(json_path, 'r') as f:
            data = json.load(f)

        filtered_images = []
        filtered_annotations = []

        class_ids_to_keep = self._get_class_ids_to_keep(data)

        for annotation in data['annotations']:
            if annotation['category_id'] in class_ids_to_keep:
                filtered_annotations.append(annotation)

        for image in data['images']:
            image_annotations = [a for a in filtered_annotations if a['image_id'] == image['id']]
            if image_annotations:
                filtered_images.append(image)
            else:
                image_path = os.path.join(dataset_path, image['file_name'])
                if os.path.exists(image_path):
                    os.remove(image_path)

        data['images'] = filtered_images
        data['annotations'] = filtered_annotations
        data['categories'] = [c for c in data['categories'] if c['name'] in self.classes_to_keep]

        with open(json_path, 'w') as f:
            json.dump(data, f)

    def _get_class_ids_to_keep(self, data):
        """
        Retrieves the class IDs to keep from the dataset based on the target classes.

        Parameters:
        -----------
        data : dict
            The loaded dataset annotation data.

        Returns:
        --------
        class_ids_to_keep : list
            List of category IDs to keep.
        """
        return [c['id'] for c in data['categories'] if c['name'] in self.classes_to_keep]

    def _register_datasets(self):
        """
        Registers the filtered datasets for training and testing in the Detectron2 framework.
        """
        register_coco_instances("fruit_dataset_train", {},
                                f"{self.train_dataset_path}/_annotations.coco.json", self.train_dataset_path)
        register_coco_instances("fruit_dataset_test", {},
                                f"{self.test_dataset_path}/_annotations.coco.json", self.test_dataset_path)

    def _setup_cfg(self):
        """
        Sets up the configuration for the Detectron2 segmentation model and training parameters.
        """
        cfg = get_cfg()
        cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
        cfg.DATASETS.TRAIN = ("fruit_dataset_train",)
        cfg.DATASETS.TEST = ("fruit_dataset_test",)
        cfg.DATALOADER.NUM_WORKERS = self.num_workers
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
        cfg.SOLVER.IMS_PER_BATCH = self.batch_size
        cfg.SOLVER.BASE_LR = self.base_lr
        cfg.SOLVER.MAX_ITER = self.max_iter
        cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
        cfg.MODEL.ROI_HEADS.NUM_CLASSES = self.num_classes
        cfg.OUTPUT_DIR = self.output_dir
        cfg.MODEL.DEVICE = self.device  # Configura el dispositivo (GPU/CPU)
        os.makedirs(self.output_dir, exist_ok=True)
        return cfg

    def show_all_classes(self):
        """
        Displays all the available classes in the dataset.
        """
        # Leer las categorías del archivo de anotaciones del conjunto de entrenamiento
        with open(f"{self.train_dataset_path}/_annotations.coco.json", 'r') as f:
            data = json.load(f)

        # Obtener las clases originales del dataset
        original_classes = [category['name'] for category in data['categories']]
        print(f"Original classes in the dataset: {original_classes}")

    def train(self, resume=False):
        """
        Starts the training process for the segmentation model.

        Parameters:
        -----------
        resume : bool
            Whether to resume training from the last checkpoint.
        """
        trainer = DefaultTrainer(self.cfg)
        trainer.resume_or_load(resume=resume)
        trainer.train()


## **Entrenamos el Modelo**

In [21]:
train_dataset_path = "train"
test_dataset_path = "valid"

# Inicializar la clase y mostrar las clases
fruit_segmentor = FruitSegmentation(train_dataset_path, test_dataset_path)
fruit_segmentor.train(resume=False)


Descargando el dataset desde Roboflow...
Dataset descargado, extraído y el archivo .zip eliminado.
[10/03 20:16:54 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[10/03 20:16:55 d2.engine.train_loop]: Starting training from iteration 0
[10/03 20:17:06 d2.utils.events]:  eta: 0:08:54  iter: 19  total_loss: 2.768  loss_cls: 1.565  loss_box_reg: 0.4752  loss_mask: 0.6887  loss_rpn_cls: 0.02185  loss_rpn_loc: 0.005474    time: 0.5389  last_time: 0.5109  data_time: 0.0342  last_data_time: 0.0276   lr: 4.9952e-05  max_mem: 3258M
[10/03 20:17:16 d2.utils.events]:  eta: 0:08:39  iter: 39  total_loss: 2.134  loss_cls: 0.9237  loss_box_reg: 0.4967  loss_mask: 0.6308  loss_rpn_cls: 0.0238  loss_rpn_loc: 0.006627    time: 0.5371  last_time: 0.5710  data_time: 0.0151  last_data_time: 0.0296   lr: 9.9902e-05  max_mem: 3258M
[10/03 20:17:27 d2.utils.events]:  eta: 0:08:32  iter: 59  total_loss: 1.663  loss_cls: 0.5627  loss_box_reg: 0.5425  loss_mask: 0.5304  loss_rpn_cls: 0.0193  loss_rpn_loc: 0.006095    time: 0.5396  last_time: 0.5606  data_time: 0.0171  last_data_time: 0.0156   lr: 0.00014985  max_mem: 3258M
[10/03 20:17:38 d2.utils.events]:  eta: 0:08:23

In [23]:
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultTrainer

# Configurar la configuración de Detectron2 para evaluación
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TEST = ("fruit_dataset_test",)  # Dataset de prueba
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "segmentacion/model_final.pth"  # Ruta de tu modelo guardado
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # Número de clases
cfg.MODEL.DEVICE = "cuda"  # O "cpu" si no tienes GPU

# Cargar el modelo entrenado sin iniciar de nuevo el entrenamiento
model = DefaultTrainer.build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

# Crear el evaluador y el dataloader de prueba
evaluator = COCOEvaluator("fruit_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "fruit_dataset_test")

# Ejecutar la evaluación y obtener las métricas
metrics = inference_on_dataset(model, val_loader, evaluator)
print(metrics)


[10/03 20:27:33 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

## **GUARDAMOS MODELO EN DRIVE**

In [24]:
import shutil

src = '/content/segmentacion'
dst = '/content/drive/MyDrive/modelos_entrenados/segmentacion'
shutil.copytree(src, dst)

'/content/drive/MyDrive/modelos_entrenados/segmentacion'

In [20]:
import shutil
shutil.rmtree('train')
shutil.rmtree('test')
shutil.rmtree('valid')
shutil.rmtree('segmentacion')